# EPSG コード


In [1]:
from pathlib import Path
from collections import Counter

from pyproj import CRS
from pyproj.database import get_authorities, get_codes
from tqdm import tqdm

## pyproj からの一覧取得


In [2]:
authorities = get_authorities()
authorities

['EPSG', 'ESRI', 'IAU_2015', 'IGNF', 'NKG', 'OGC', 'PROJ']

In [3]:
epsg_codes = {}

for code in tqdm(get_codes("EPSG", "CRS")):
    crs = CRS.from_epsg(code)
    epsg_codes[code] = {
        "name": crs.name,
        "area_of_use": crs.area_of_use.name,
        "coordinate_operation": (
            crs.coordinate_operation.name if crs.coordinate_operation else None
        ),
        "datum": crs.datum.name if crs.datum else None,
    }

epsg_codes

100%|██████████| 6934/6934 [00:03<00:00, 2068.25it/s]


{'10150': {'name': 'MSL UK & Ireland VORF08 depth',
  'area_of_use': 'Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshore, nearshore and offshore.',
  'coordinate_operation': None,
  'datum': 'Mean Sea Level UK & Ireland VORF08'},
 '10151': {'name': 'CD UK & Ireland VORF08 depth',
  'area_of_use': 'Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshore, nearshore and offshore.',
  'coordinate_operation': None,
  'datum': 'Chart Datum UK & Ireland VORF08'},
 '10156': {'name': 'ETRS89 + MSL UK & Ireland VORF08 depth',
  'area_of_use': 'Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshore, nearshore and offshore.',
  'coordinate_operation': None,
  'datum': 'European Terrestrial Reference System 1989 ensemble'},
 '10157': {'name': 'ETRS89 + CD UK & Ireland VORF08 depth',
  'area_of_use': 'Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshore, nearshore and offs

In [4]:
epsg_codes["4326"]

{'name': 'WGS 84',
 'area_of_use': 'World.',
 'coordinate_operation': None,
 'datum': 'World Geodetic System 1984 ensemble'}

In [5]:
epsg_codes["10162"]

{'name': 'JGD2011 / Japan Plane Rectangular CS I + JGD2011 (vertical) height',
 'area_of_use': 'Japan - onshore - Kyushu west of approximately 130°E - Nagasaki-ken',
 'coordinate_operation': None,
 'datum': 'Japanese Geodetic Datum 2011'}

In [6]:
import json

with open("./data/epsg.json", "w") as f:
    json.dump(epsg_codes, f, ensure_ascii=False, indent=4)

In [7]:
!head ./data/epsg.json

{
    "10150": {
        "name": "MSL UK & Ireland VORF08 depth",
        "area_of_use": "Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshore, nearshore and offshore.",
        "coordinate_operation": null,
        "datum": "Mean Sea Level UK & Ireland VORF08"
    },
    "10151": {
        "name": "CD UK & Ireland VORF08 depth",
        "area_of_use": "Ireland and United Kingdom (UK) (including Isle of Man and Channel Islands) - inshore, nearshore and offshore.",


---


## Appendix: epsg.org データセットからの取得

ソース: https://epsg.org/home.html


In [8]:
wkt_paths = sorted(Path("./data/epsg/EPSG-v11_008-WKT/").glob("*.wkt"))
len(wkt_paths), wkt_paths[:5]

(9698,
 [PosixPath('data/epsg/EPSG-v11_008-WKT/EPSG-CRS-10150.wkt'),
  PosixPath('data/epsg/EPSG-v11_008-WKT/EPSG-CRS-10151.wkt'),
  PosixPath('data/epsg/EPSG-v11_008-WKT/EPSG-CRS-10156.wkt'),
  PosixPath('data/epsg/EPSG-v11_008-WKT/EPSG-CRS-10157.wkt'),
  PosixPath('data/epsg/EPSG-v11_008-WKT/EPSG-CRS-10158.wkt')])

### EPSG コードから pyproj で CRS を作成


In [9]:
epsg_codes = {}
invalid_epsg_codes = []
invalid_epsg_error_msgs = []

for p in tqdm(wkt_paths):
    stems = p.stem.split("-")
    assert stems[0] == "EPSG"
    epsg_code = int(stems[-1])

    try:
        crs = CRS.from_epsg(epsg_code)
        crs_name = crs.name
        crs_epsg = str(crs.to_epsg())
        epsg_codes[crs_epsg] = crs_name
    except Exception as e:
        invalid_epsg_codes.append(p)
        invalid_epsg_error_msgs.append(str(e))
        continue

len(epsg_codes), len(invalid_epsg_codes)

100%|██████████| 9698/9698 [00:10<00:00, 890.88it/s] 


(6936, 2762)

In [10]:
set([msg.split("(")[1] for msg in invalid_epsg_error_msgs])

{'Internal Proj Error: proj_create: crs not found)'}

### WKT ファイルから読み込み


In [11]:
epsg_codes = {}
invalid_epsg_codes = []
invalid_epsg_error_msgs = []


for p in tqdm(wkt_paths):
    with open(p) as f:
        wkt_text = f.read()
    try:
        crs = CRS.from_wkt(wkt_text)
    except Exception as e:
        invalid_epsg_codes.append(p)
        invalid_epsg_error_msgs.append(str(e))
        continue

    crs_name = crs.name
    crs_epsg = str(crs.to_epsg())
    epsg_codes[crs_epsg] = crs_name

len(epsg_codes), len(invalid_epsg_codes)

100%|██████████| 9698/9698 [00:09<00:00, 1070.59it/s]


(6550, 2690)

In [12]:
epsg_codes["10162"]

'JGD2011 / Japan Plane Rectangular CS I + JGD2011 (vertical) height'

In [13]:
epsg_codes["4326"]

'WGS 84'

### エラーの確認


In [14]:
Counter([msg.split(":")[0] for msg in invalid_epsg_error_msgs])

Counter({'Input is not a CRS': 2677,
         'Invalid projection': 8,
         'Invalid WKT string': 5})

In [15]:
# "Input is not a CRS"
Counter(
    [
        msg.split("[")[0]
        for msg in invalid_epsg_error_msgs
        if msg.startswith("Input is not a CRS:")
    ]
)

Counter({'Input is not a CRS: COORDINATEOPERATION': 2491,
         'Input is not a CRS: CONCATENATEDOPERATION': 186})

In [16]:
# "Invalid projection"
Counter(
    [
        msg.split("[")[0]
        for msg in invalid_epsg_error_msgs
        if msg.startswith("Invalid projection:")
    ]
)

Counter({'Invalid projection: DERIVEDPROJECTED': 4,
         'Invalid projection: POINTMOTIONOPERATION': 3,
         'Invalid projection: CONCATENATEDOPERATION': 1})

In [17]:
# "Invalid WKT string"
Counter(
    [msg for msg in invalid_epsg_error_msgs if msg.startswith("Invalid WKT string")]
)

Counter({'Invalid WKT string: WKT is not supported for vertical CRS derived in multiple steps.': 4,
         'Invalid WKT string: POINTMOTIONOPERATION["New Caledonia velocity model 2015",SOURCECRS[GEODCRS["RGNC15",DATUM["Reseau Geodesique de Nouvelle Caledonie 2015",ELLIPSOID["GRS 1980",6378137,298.257222101,LENGTHUNIT["metre",1,ID["EPSG",9001]],ID["EPSG",7019]],ID["EPSG",1357]],CS[Cartesian,3,ID["EPSG",6500]],AXIS["Geocentric X (X)",geocentricX,LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["Geocentric Y (Y)",geocentricY,LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["Geocentric Z (Z)",geocentricZ,LENGTHUNIT["metre",1,ID["EPSG",9001]]],ID["EPSG",10308]]],METHOD["Point motion (geocen) by grid (BGN)",ID["EPSG",1120]],PARAMETERFILE["Point motion velocity grid file","grmmncl08.tac",ID["EPSG",1050]],PARAMETER["EPSG code for Interpolation CRS",10312,ID["EPSG",1048]],OPERATIONACCURACY[0.075],ID["EPSG",10323]]': 1})